In [3]:
import numpy as np
import cvxpy as cp

In [89]:
def poly_cvar_ball(p,alpha,x,f,r):   #### Function for Calculating Robust CVaR with Polynomial Divergence
    n = len(x)
    theta_1 = cp.Variable(1)
    theta_2 = cp.Variable(1)
    lbda = cp.Variable(1, nonneg = True)
    t = cp.Variable(n)
    z = cp.Variable(n, nonneg= True)
    s = cp.Variable(n)
    constraints = [lbda-s*(1-p) <= z, theta_1 + cp.pos((1 / alpha) * (theta_2 - x)) <= s]
    macht = np.array([(p-1)/p, 1-(p-1)/p])
    for i in range(n):
        constraints.append(z[i] <= cp.geo_mean(cp.vstack([lbda+p*t[i],lbda]),macht)) 
    obj = cp.Minimize(-theta_1-theta_2+lbda*r+t@f)
    prob = cp.Problem(obj,constraints)
    prob.solve(solver = cp.MOSEK)
    return(prob.value, theta_1.value, theta_2.value, lbda.value)


def poly_cvar_ball_IS(p, alpha, x, f, r):   #### With importance sampling
    n = len(x)
    theta_1 = cp.Variable(1)
    theta_2 = cp.Variable(1)
    lbda = cp.Variable(1, nonneg = True)
    t = cp.Variable(n)
    z = cp.Variable(n, nonneg= True)
    s = cp.Variable(n)
    constraints = [lbda-s*(1-p) <= z, theta_1 + cp.pos((1 / alpha) * (theta_2 - x)) <= s]
    macht = np.array([(p-1)/p, 1-(p-1)/p])
    for i in range(n):
        constraints.append(z[i] <= cp.geo_mean(cp.vstack([lbda+p*t[i],lbda]),macht)) 
    #### multiplying weighting factor 2.2/|x|**1.2
    obj = cp.Minimize(-theta_1-theta_2+lbda*r+t@(cp.multiply(f,2.2/(np.abs(x))**1.2)))
    prob = cp.Problem(obj,constraints)
    prob.solve(solver = cp.MOSEK)
    return(prob.value, theta_1.value, theta_2.value, lbda.value)

def kl_cvar_ball(alpha, x, f, r):  #### Function for Calculating Robust CVaR with KL-Divergence
    n = len(x)
    theta_1 = cp.Variable()
    theta_2 = cp.Variable()
    lbda = cp.Variable(nonneg=True)
    t = cp.Variable(n)
    w = cp.Variable(n)
    s = cp.Variable(n)
    constraints = []
    constraints.append(theta_1 + cp.pos(1 / alpha * (theta_2 - x)) <= s)
    constraints.append(w - lbda <= t)
    constraints.append(cp.kl_div(lbda, w) + lbda + s - w <= 0)
    obj = cp.Minimize(-theta_1 - theta_2 + lbda * r + cp.sum(cp.multiply(t, f)))
    prob = cp.Problem(obj, constraints)
    prob.solve(solver=cp.MOSEK)
    return prob.value, theta_1.value, theta_2.value, lbda.value

def kl_cvar_ball_IS(alpha, x, f, r):   #### With importance sampling
    n = len(x)
    theta_1 = cp.Variable()
    theta_2 = cp.Variable()
    lbda = cp.Variable(nonneg=True)
    t = cp.Variable(n)
    w = cp.Variable(n)
    s = cp.Variable(n)
    constraints = []
    constraints.append(theta_1 + cp.pos(1 / alpha * (theta_2 - x)) <= s)
    constraints.append(w - lbda <= t)
    constraints.append(cp.kl_div(lbda, w) + lbda + s - w <= 0)
    #### multiplying weighting factor 2.2/|x|**1.2
    obj = cp.Minimize(-theta_1 - theta_2 + lbda * r + cp.sum(cp.multiply(t, cp.multiply(f,2.2/(np.abs(x))**1.2))))
    prob = cp.Problem(obj, constraints)
    prob.solve(solver=cp.MOSEK)
    return prob.value, theta_1.value, theta_2.value, lbda.value

In [13]:
#### Exact CVaR_{alpha} of Pareto with scale x_m, shape p_0, according to Norton et al. [2021] 
                            
def CVaR_Pareto(x_m,p_0,alpha):       
    return((x_m*p_0)/((1-alpha)**(1/p_0)*(p_0-1)))

In [73]:
rcvar_poly1 = []
rcvar_poly2 = []
n = 500
for r in [0.001, 0.005, 0.01,0.02, 0.05,0.1]:
    np.random.seed(1)
    rcvar_val1 = []
    rcvar_val2 = []
    for j in range(10):
        X = -(np.random.pareto(2.2,size = n )+1)
        val1 = poly_cvar_ball(3,0.025,X,np.zeros(n)+1/n,r)[0]
        val2 = poly_cvar_ball(11,0.025,X,np.zeros(n)+1/n,r)[0]
        rcvar_val1.append(val1)
        rcvar_val2.append(val2)
        print(r,val1,val2)
    rcvar_poly1.append(np.median(np.array(rcvar_val1)))
    rcvar_poly2.append(np.median(np.array(rcvar_val2)))
print(rcvar_poly1)
print(rcvar_poly2)
                   
        
    
    

0.001 8.59343398574143 8.288481790556704
0.001 8.805980821121612 8.540467565483738
0.001 9.958876335466902 9.476761872814446
0.001 9.677948795492341 9.196353257196126
0.001 7.81009494595289 7.322874910029405
0.001 15.76779122687471 13.99957834488108
0.001 10.492082495645274 10.033545871905313
0.001 9.21141316617928 8.889391766636397
0.001 10.88011671052391 10.480862928869678
0.001 8.710469007571715 8.463382026038554
0.005 9.465414380772208 8.658195711957728
0.005 9.58210298916561 8.851342222996772
0.005 11.349791252553267 9.986731415496482
0.005 10.969113947594852 9.624065100618376
0.005 9.023350141286388 7.711698312951141
0.005 19.839480969487617 15.204284091332042
0.005 11.815604680569379 10.528651273548151
0.005 10.208617957286798 9.272126562528367
0.005 12.167088123946712 11.038165873315236
0.005 9.502033393127522 8.790053614780021
0.01 9.987213244819879 8.833016100952893
0.01 10.055732395026851 9.00292921082664
0.01 12.242728191182072 10.241555433800215
0.01 11.830446070125834 9.8

In [77]:
np.random.seed(1)    
N = np.arange(500,6500,500)
poly_r = np.zeros(len(N))
kl_r = np.zeros(len(N))
X = -(np.random.pareto(2.2,size = 6500 )+1)
for i in range(len(N)):
    n = N[i]
    poly_r[i] = poly_cvar_ball(3,0.025,X[0:n],np.zeros(n)+1/n,0.05)[0]
    kl_r[i] = kl_cvar_ball(1-0.975,X[0:n],np.zeros(n)+1/n,0.05)[0]
    print('n:', n, 'Polynomial:',poly_r[i], 'KL:',kl_r[i])

n: 500 Polynomial: 11.388443347606318 KL: 14.765213250325289
n: 1000 Polynomial: 11.451656615538742 KL: 14.649903249354185
n: 1500 Polynomial: 12.7475609177787 KL: 18.348995480095567
n: 2000 Polynomial: 13.29809832684494 KL: 18.774649053258525
n: 2500 Polynomial: 13.250239052446926 KL: 19.263968520450724
n: 3000 Polynomial: 17.438627268985872 KL: 44.461492653158594
n: 3500 Polynomial: 17.201554547180823 KL: 43.34263554980977
n: 4000 Polynomial: 16.712882127140578 KL: 42.26719869079084
n: 4500 Polynomial: 16.611283210552966 KL: 41.50727975423438
n: 5000 Polynomial: 16.16050910418413 KL: 40.679933572729716
n: 5500 Polynomial: 16.305916382275257 KL: 40.219249386464476
n: 6000 Polynomial: 15.966603677880212 KL: 39.579650366396976


In [87]:
np.random.seed(1)                        #### Same experiment, but with importance sampling
N = np.arange(500,6500,500)
poly_r_IS = np.zeros(len(N))
kl_r_IS = np.zeros(len(N))
X = -(np.random.pareto(1,size = 6500)+1)
for i in range(len(N)):
    n = N[i]
    poly_r_IS[i] = poly_cvar_ball_IS(3,0.025,X[0:n],np.zeros(n)+1/n,0.05)[0]
    kl_r_IS[i] = kl_cvar_ball_IS(1-0.975,X[0:n],np.zeros(n)+1/n,0.05)[0]
    print('n:', n, 'Polynomial:',poly_r_IS[i], 'KL:',kl_r_IS[i])

n: 500 Polynomial: 21.869633343140663 KL: 122.42297583874426
n: 1000 Polynomial: 21.470565839094014 KL: 116.57695288257328
n: 1500 Polynomial: 21.910822270640427 KL: 174.9998940110524
n: 2000 Polynomial: 22.27147553688856 KL: 178.6241345843278
n: 2500 Polynomial: 21.377458954155152 KL: 186.41426148177223
n: 3000 Polynomial: 21.547621139516497 KL: 1400.8890588046688
n: 3500 Polynomial: 21.693272973167872 KL: 1384.1704131400984
n: 4000 Polynomial: 21.649056457942713 KL: 1369.865730693035
n: 4500 Polynomial: 21.665561329093546 KL: 1356.5008321908022
n: 5000 Polynomial: 21.66863343310711 KL: 1346.4872063293737
n: 5500 Polynomial: 21.702560108224635 KL: 1336.8319943622284
n: 6000 Polynomial: 21.796308676434673 KL: 1328.152964477336
